<a href="https://colab.research.google.com/github/kmalhotra18/Unit-Test-Generator-Product/blob/main/Generate_Unit_Test_Cases_Product_Open_Source_Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Product that will **generate** **Unit Test cases**

In [ ]:
# Step 1: Install necessary packages
!pip install -q openai gradio transformers accelerate

In [ ]:
# STEP 2: Imports
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from openai import OpenAI
from google.colab import userdata
import requests

In [ ]:
# STEP 3: Load API keys from Colab secrets
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
HUGGINGFACE_API_KEY = userdata.get("HF_TOKEN")

In [ ]:
# Qwen endpoint info
CODE_QWEN_ID = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://ebr1diyv7rshsjxu.us-east-1.aws.endpoints.huggingface.cloud"  # Your Hugging Face Inference Endpoint

In [ ]:
# STEP 4: Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# def load_qwen():
#     global qwen_model, qwen_tokenizer
#     if qwen_model is None or qwen_tokenizer is None:
#         qwen_tokenizer = AutoTokenizer.from_pretrained(
#             "Qwen/CodeQwen1.5-7B-Chat",
#             token=HUGGINGFACE_API_KEY
#         )
#         qwen_model = AutoModelForCausalLM.from_pretrained(
#             "Qwen/CodeQwen1.5-7B-Chat",
#             torch_dtype="auto",
#             device_map="auto",
#             token=HUGGINGFACE_API_KEY
#         )

In [ ]:
# STEP 6: Unified function to generate test cases - Unit test generation logic for both models
def generate_unit_tests(code: str, model_choice: str) -> str:
    prompt = f"""You are an expert Python developer. Based on the following code, generate complete and concise unit test cases using the unittest module in Python.\n\nCode:\n{code}\n\nUnit Tests:"""

    try:
        if model_choice == "GPT-4o (OpenAI)":
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=800,
            )
            return response.choices[0].message.content

        elif model_choice == "CodeQwen1.5-7B-Chat (HuggingFace Endpoint)":
            headers = {
                "Authorization": f"Bearer {HUGGINGFACE_API_KEY}",
                "Content-Type": "application/json"
            }
            payload = {
                "inputs": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                "parameters": {
                    "max_new_tokens": 512,
                    "temperature": 0.3
                }
            }

            response = requests.post(CODE_QWEN_URL, headers=headers, json=payload)
            if response.status_code == 200:
                output = response.json()
                return output[0]["generated_text"].split(prompt)[-1].strip()
            else:
                return f"Hugging Face API error: {response.status_code} - {response.text}"

        else:
            return "Unsupported model selection."

    except Exception as e:
        return f"Error generating unit tests:\n\n{str(e)}"

In [ ]:
# Step 4: Gradio interface

def handle_input(code_input, file_input, model_choice):
    if file_input is not None:
        code = file_input.read().decode("utf-8")
    elif code_input.strip():
        code = code_input
    else:
        return "Please paste code or upload a Python file."
    return generate_unit_tests(code, model_choice)

with gr.Blocks() as demo:
    gr.Markdown("## 🧪 Python Unit Test Generator")
    gr.Markdown("Choose a model, paste your code or upload a `.py` file, and click **Generate**.")

    model_dropdown = gr.Dropdown(
        ["GPT-4o (OpenAI)", "CodeQwen1.5-7B-Chat (HuggingFace Endpoint)"],
        label="Choose Model",
        value="GPT-4o (OpenAI)"
    )

    with gr.Row():
        code_input = gr.Textbox(lines=20, label="Paste Your Python Code")
        file_input = gr.File(label="Or Upload `.py` File", file_types=[".py"])

    generate_btn = gr.Button("Generate Unit Tests")
    output = gr.Textbox(label="Generated Unit Tests", lines=20)

    generate_btn.click(
        fn=handle_input,
        inputs=[code_input, file_input, model_dropdown],
        outputs=output
    )

demo.launch()

In [ ]:
from google.colab import userdata
print(userdata.get("HF_TOKEN"))


In [ ]:
import requests
from google.colab import userdata

HUGGINGFACE_API_KEY = userdata.get("HF_TOKEN")
CODE_QWEN_URL = "https://ebr1diyv7rshsjxu.us-east-1.aws.endpoints.huggingface.cloud"

headers = {
    "Authorization": f"Bearer {HUGGINGFACE_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "inputs": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a function to add two numbers in Python."}
    ],
    "parameters": {
        "max_new_tokens": 100
    }
}

response = requests.post(CODE_QWEN_URL, headers=headers, json=payload)
print("Status:", response.status_code)
print("Response:", response.text)
